# Components

In [28]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name = "gpt-4o-mini"

sys_prompt = '''
Summarize or rewrite the following text using dashes to list key points and crucial details, like data and news. 
Avoid duplicate points, and ensure the context is preserved by not over-simplifying or over-summarizing sentences. 
Return the result in raw text format.
'''

def ask_gpt(prompt: str) -> str:
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {
                "role": "system", 
                "content": sys_prompt
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        max_tokens=4096
    )
    return response.choices[0].message.content

def logging(filename: str, content: str):
    with open(f"{filename}.txt", "w", encoding="utf-8") as file:
        file.write(content)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def chunking(transcript: str): # tokens
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        model_name="gpt-4o", 
        chunk_size=5000,
        chunk_overlap=500
    )
    return text_splitter.split_text(transcript)

def summarize(chapters):
    concatenate = ""
    for chapter in chapters:
        summary = ask_gpt(chapter)
        concatenate = concatenate + "\n" + summary
    return summary

transcript = ""
with open("transcript.txt") as f:
    transcript = f.read()

while True:
    chapters = chunking(transcript)
    transcript = summarize(chapters)
    if len(chapters) == 1:
        break

logging("result", transcript)